# Setup Imports:

In [18]:
#import necessary packages:
import os
from glob import glob
#from pre_processing_IOV import *
from sklearn.model_selection import train_test_split
import my_transforms
import torchvision.transforms.functional as F
import numpy as np
import torchvision
import sklearn
import pandas as pd
import monai
from monai.metrics import DiceMetric
import torch
import matplotlib.pyplot as plt
from tensorflow import keras
from statsmodels.stats.inter_rater import fleiss_kappa
import cv2
import InterOperator_utils
import pre_processing

#### Step 1: Load model and data

In [2]:
#set data loading parameters:
use_AMF_images = False
use_Inpaint_images = False 

#determine whether to preprocess the data:
apply_AMF_flag = False

#set model parameters:
rand_state = 42 #random state for the train/test split
training_channels = 1
batch_size= 16
augmentation_flag = False
n_classes = 5
aug_params = {
    'RandAdjustContrastd' : {'prob':1.0, 'gamma':(0.5, 2.5), 'allow_missing_keys':False},
    'GaussianSmoothd' : {'sigma' : 1},
    'CustomGaussianNoised': {},
    'Rotated_180': {'angle': np.pi, 'mode': ["bilinear", "nearest"]},
    'CenterSpatialCropd_200': {'roi_size': (200,200)},
    'RandZoomd': {'prob': 1, 'min_zoom': 0.8, 'max_zoom': 0.9, 'mode': ["area", "nearest"]},
}
aug_transformations = my_transforms.make_aug_transform(aug_params)
n_aug_transformations = len(list(aug_transformations))
dice_metric = DiceMetric(include_background=True, reduction="mean") #to_onehot = True or False


# ---------------------------------------------------------------------------------------------------------------------------- #
# STEP 0:
# ---------------------------------------------------------------------------------------------------------------------------- #
print('Step 1 is running: Access images from file paths')
# ---------------------------------------------------------------------------------------------------------------------------- #
# Access images from file paths:
# ---------------------------------------------------------------------------------------------------------------------------- #

#define folder where images are stored:
date_labelbox_pull = '20220625' #image folder named by pull date - if not pulling data from labelbox in this run
CLEAR_root_path = '/home/obsegment/code/ResearchDataset/CLEAR'
raw_img_path = os.path.join(CLEAR_root_path, 'data')
results_folder = os.path.join(CLEAR_root_path, 'Results')
   

data_path = None
if use_AMF_images:
    start_AMF_path = os.path.join(raw_img_path, date_labelbox_pull)
    data_path = start_AMF_path
    img_prefix = '*AMF_*'
elif use_Inpaint_images:
    start_inpaint_path = os.path.join(raw_img_path, date_labelbox_pull)
    data_path = start_inpaint_path
    img_prefix = '*Inpaint_*'
else:
    img_prefix = 'CLEAR_g[0-9]_im*' #look for this expression only in the beginning of the path
    data_path = os.path.join(raw_img_path, date_labelbox_pull)
    print('No path provided for input images, check if this path is correct: ' + data_path)

print("Starting paths have been saved.")

#generate list of images and segmentations for each expert:
images = sorted(glob(os.path.join(data_path, img_prefix))) #list of image paths
KJ_segs = sorted(glob(os.path.join(data_path, "*lab[0-9][0-9][0-9]_KJ*"))) #list of mask png file paths for expert KJ
DC_segs = sorted(glob(os.path.join(data_path, "*lab[0-9][0-9][0-9]_DC*"))) #list of mask png file paths for expert DC
MH_segs = sorted(glob(os.path.join(data_path, "*lab[0-9][0-9][0-9]_MH*"))) #list of mask png file paths for expert MH
maj_segs = sorted(glob(os.path.join(data_path, "*lab[0-9][0-9][0-9]_majGT*"))) #list of mask file paths for ground truth majority vote

#print the number of items in each image and segmentation list - they may be different lengths:
print("there are {} images in the starting directory".format(len(images)))
print("there are {} GT segmentations in the majority starting directory".format(len(maj_segs)))
print("there are {} segmentations in the KJ starting directory".format(len(KJ_segs)))
print("there are {} segmentations in the DC starting directory".format(len(DC_segs)))
print("there are {} segmentations in the MH starting directory".format(len(MH_segs)))

Step 1 is running: Access images from file paths
No path provided for input images, check if this path is correct: /home/obsegment/code/ResearchDataset/CLEAR/data/20220625
Starting paths have been saved.
there are 250 images in the starting directory
there are 246 GT segmentations in the majority starting directory
there are 250 segmentations in the KJ starting directory
there are 248 segmentations in the DC starting directory
there are 246 segmentations in the MH starting directory


#### Step 2: Only include images that have been labeled by each expert


In [3]:
#create empty lists to fill with image and label IDs for each expert
image_id_list = []
expert_seg_id_lists = [[], [], []]

#from list of image path, get image ID, and generate label ID list for each expert:
#only add image/segmentation to ID list if corresponding label_id exists in list of majority GT segmentations
for s, seg in enumerate(maj_segs):
    for i, img in enumerate(images):
        image_id = img.split('/')[-1].split('.')[0] #take everything before .png and after the folder path
        label_id = image_id.replace('im', 'lab')
        if use_Inpaint_images:
            label_id = label_id.replace("Inpaint_", "")
        if label_id in seg:
            image_id_list.append(img)
            
    for i, expert_segs in enumerate([KJ_segs, DC_segs, MH_segs]):
        for idx, lab in enumerate(expert_segs):
            i_label_id = lab.split('/')[-1].split('.')[0]
            i_label_id = i_label_id.replace('KJ', '').replace('MH', '').replace('DC', '')
            if i_label_id in seg:
                expert_seg_id_lists[i].append(lab)
                
KJ_seg_id_list = expert_seg_id_lists[0]
DC_seg_id_list = expert_seg_id_lists[1]
MH_seg_id_list = expert_seg_id_lists[2]      


#print the number of items in each image and segmentation list - check that all lists have same length:      
print("length of image_id_list is {}".format(len(image_id_list)))
print("length of KJ segmentation list  is {}".format(len(KJ_seg_id_list)))
print("length of DC segmentation list  is {}".format(len(DC_seg_id_list)))
print("length of MH segmentation list  is {}".format(len(MH_seg_id_list)))
print("length of segmentation list  is {}".format(len(maj_segs)))

#display the path for one index to verify that the naming conventions align:
verify_index = 12
print(image_id_list[verify_index])
print(KJ_seg_id_list[verify_index])
print(DC_seg_id_list[verify_index])
print(MH_seg_id_list[verify_index])

length of image_id_list is 246
length of KJ segmentation list  is 246
length of DC segmentation list  is 246
length of MH segmentation list  is 246
length of segmentation list  is 246
/home/obsegment/code/ResearchDataset/CLEAR/data/20220625/CLEAR_g6_im015.png
/home/obsegment/code/ResearchDataset/CLEAR/data/20220625/CLEAR_g6_lab015_KJ.png
/home/obsegment/code/ResearchDataset/CLEAR/data/20220625/CLEAR_g6_lab015_DC.png
/home/obsegment/code/ResearchDataset/CLEAR/data/20220625/CLEAR_g6_lab015_MH.png


#### Step 3: Separate images and segmentations based on quality score (g6, g8, g9) & load the data

In [12]:
#create empty image and segmentation lists for each grade:
g6_images, g8_images, g9_images = ([] for i in range(3))
g6_maj_segs, g8_maj_segs, g9_maj_segs = ([] for i in range(3))
g6_KJ_segs, g8_KJ_segs, g9_KJ_segs = ([] for i in range(3))
g6_DC_segs, g8_DC_segs, g9_DC_segs = ([] for i in range(3))
g6_MH_segs, g8_MH_segs, g9_MH_segs = ([] for i in range(3))

#subdivide image and segmentation id lists based on CLEAR grading scores:
for idx, seg in enumerate(maj_segs):
    if 'g6' in seg:           
        g6_images.append(image_id_list[idx])
        g6_maj_segs.append(maj_segs[idx])
        g6_KJ_segs.append(KJ_seg_id_list[idx])
        g6_DC_segs.append(DC_seg_id_list[idx])
        g6_MH_segs.append(MH_seg_id_list[idx])
    elif 'g8' in seg:
        g8_images.append(image_id_list[idx])
        g8_maj_segs.append(maj_segs[idx])
        g8_KJ_segs.append(KJ_seg_id_list[idx])
        g8_DC_segs.append(DC_seg_id_list[idx])
        g8_MH_segs.append(MH_seg_id_list[idx])
    elif 'g9' in seg:
        g9_images.append(image_id_list[idx])
        g9_maj_segs.append(maj_segs[idx])
        g9_KJ_segs.append(KJ_seg_id_list[idx])
        g9_DC_segs.append(DC_seg_id_list[idx])
        g9_MH_segs.append(MH_seg_id_list[idx])

#create 3 groupings of data paths, before splitting data set:
g6_data_paths = [{"image": img, "label": {"KJ": KJ_seg, "DC": DC_seg, "MH": MH_seg, "maj": maj_seg}} for img, KJ_seg, DC_seg, MH_seg, maj_seg in zip(g6_images, g6_KJ_segs, g6_DC_segs, g6_MH_segs, g6_maj_segs)] 
g8_data_paths = [{"image": img, "label": {"KJ": KJ_seg, "DC": DC_seg, "MH": MH_seg, "maj": maj_seg}} for img, KJ_seg, DC_seg, MH_seg, maj_seg in zip(g8_images, g8_KJ_segs, g8_DC_segs, g8_MH_segs, g8_maj_segs)] 
g9_data_paths = [{"image": img, "label": {"KJ": KJ_seg, "DC": DC_seg, "MH": MH_seg, "maj": maj_seg}} for img, KJ_seg, DC_seg, MH_seg, maj_seg in zip(g9_images, g9_KJ_segs, g9_DC_segs, g9_MH_segs, g9_maj_segs)] 

# ---------------------------------------------------------------------------------------------------------------------------- #
# Open image/labels as numpy arrays, convert label to class value     
# ---------------------------------------------------------------------------------------------------------------------------- #
#load numpy arrays from image/segmentation paths
g6_np_data_files = InterOperator_utils.convert_paths_to_np_preserveFileName(g6_data_paths, training_channels)
g8_np_data_files = InterOperator_utils.convert_paths_to_np_preserveFileName(g8_data_paths, training_channels)
g9_np_data_files = InterOperator_utils.convert_paths_to_np_preserveFileName(g9_data_paths, training_channels)

# Pre-process images - this function expects numpy inputs
g6_class_json_dict = InterOperator_utils.ToClass_fnc_preserveFileName(g6_np_data_files)
g8_class_json_dict = InterOperator_utils.ToClass_fnc_preserveFileName(g8_np_data_files)
g9_class_json_dict = InterOperator_utils.ToClass_fnc_preserveFileName(g9_np_data_files)


Note: The above section takes roughly 30 minutes to run.

### Step 4: Split data into train/validation/test sets
##### This is done in a way that preserves the proportions of g6, g8, and g9 data in the original dataset.

In [16]:
#split the data into train and validation files:
g6_train_files, g6_tmp_files = train_test_split(g6_class_json_dict, test_size = 0.3, random_state = rand_state)
g8_train_files, g8_tmp_files = train_test_split(g8_class_json_dict, test_size = 0.3, random_state = rand_state)
g9_train_files, g9_tmp_files = train_test_split(g9_class_json_dict, test_size = 0.3, random_state = rand_state)
#further split the validation set into validation and test data:
g6_val_files, g6_test_files = train_test_split(g6_tmp_files, test_size = 0.33, random_state = rand_state)
g8_val_files, g8_test_files = train_test_split(g8_tmp_files, test_size = 0.33, random_state = rand_state)
g9_val_files, g9_test_files = train_test_split(g9_tmp_files, test_size = 0.33, random_state = rand_state)
#combine lists for each group: train, validation and test:
train_files = g6_train_files + g8_train_files + g9_train_files
val_files = g6_val_files + g8_val_files + g9_val_files
test_files = g6_test_files + g8_test_files + g9_test_files
#create list of all files in the dataset:
all_files = train_files + val_files + test_files

##### Step 5: Extract file names for each dataset. Preserve only image and label dictionary keys when loading the data for model training.

In [21]:
#Extract file names for all files:
all_filenames = []
KJ_all_files_data = []
DC_all_files_data = []
MH_all_files_data = []
maj_all_files_data = []

#Create dictionary of image and labels only, separate filenames into list:
for dict in all_files:
    all_filenames.append(dict["filename"])
    KJ_all_files_data.append({"image": dict["image"], "label": dict["label"]["KJ"]})
    DC_all_files_data.append({"image": dict["image"], "label": dict["label"]["DC"]})
    MH_all_files_data.append({"image": dict["image"], "label": dict["label"]["MH"]})
    maj_all_files_data.append({"image": dict["image"], "label": dict["label"]["maj"]})

#Extract file names for train files:
train_filenames = []
KJ_train_files_data = []
DC_train_files_data = []
MH_train_files_data = []
maj_train_files_data = []
for dict in train_files:
    train_filenames.append(dict["filename"])
    KJ_train_files_data.append({"image": dict["image"], "label": dict["label"]["KJ"]})
    DC_train_files_data.append({"image": dict["image"], "label": dict["label"]["DC"]})
    MH_train_files_data.append({"image": dict["image"], "label": dict["label"]["MH"]})
    maj_train_files_data.append({"image": dict["image"], "label": dict["label"]["maj"]})

#Extract file names for validation files:
val_filenames = []
val_files_data = []
KJ_val_files_data = []
DC_val_files_data = []
MH_val_files_data = []
maj_val_files_data = []
for dict in val_files:
    val_filenames.append(dict["filename"])
    KJ_val_files_data.append({"image": dict["image"], "label": dict["label"]["KJ"]})
    DC_val_files_data.append({"image": dict["image"], "label": dict["label"]["DC"]})
    MH_val_files_data.append({"image": dict["image"], "label": dict["label"]["MH"]})
    maj_val_files_data.append({"image": dict["image"], "label": dict["label"]["maj"]})

#Extract file names for test files:
test_filenames = []
test_files_data = []
KJ_test_files_data = []
DC_test_files_data = []
MH_test_files_data = []
maj_test_files_data = []
for dict in test_files:
    test_filenames.append(dict["filename"])
    test_files_data.append({"image": dict["image"], "label": dict["label"]})
    KJ_test_files_data.append({"image": dict["image"], "label": dict["label"]["KJ"]})
    DC_test_files_data.append({"image": dict["image"], "label": dict["label"]["DC"]})
    MH_test_files_data.append({"image": dict["image"], "label": dict["label"]["MH"]})
    maj_test_files_data.append({"image": dict["image"], "label": dict["label"]["maj"]})

#review composition of train/val/test dataset:
n_train_files = len(train_files)
n_val_files = len(val_files)
n_test_files = len(test_files)
n_total_files = n_train_files + n_val_files + n_test_files
#double check that the length of all files is the same as n_total_files
if n_total_files == len(all_files):
    print("There are {} files in total".format(n_total_files))
    
#print the number of files in each dataset
print('n_train_files = ' + str(n_train_files))
print('n_val_files = ' + str(n_val_files))
print('n_test_files = ' + str(n_test_files))

#print the percentage of files in each dataset:
percent_train = int(np.round(n_train_files/n_total_files*100,0))
percent_val = int(np.round(n_val_files/n_total_files*100,0))
percent_test = int(np.round(n_test_files/n_total_files*100,0))
print("The training set is {} percent of the dataset, the validation set is {} percent of the dataset and the test set is {} percent of the test set".format(percent_train,percent_val,percent_test))

#load data from file paths:
KJ_train_loader, KJ_val_loader, KJ_test_loader, KJ_train_ds, KJ_val_ds, KJ_test_ds= pre_processing.load_data(KJ_train_files_data, KJ_val_files_data, KJ_test_files_data, batch_size, augmentation_flag, aug_transformations)
DC_train_loader, DC_val_loader, DC_test_loader, DC_train_ds, DC_val_ds, DC_test_ds= pre_processing.load_data(DC_train_files_data, DC_val_files_data, DC_test_files_data, batch_size, augmentation_flag, aug_transformations)
MH_train_loader, MH_val_loader, MH_test_loader, MH_train_ds, MH_val_ds, MH_test_ds= pre_processing.load_data(MH_train_files_data, MH_val_files_data, MH_test_files_data, batch_size, augmentation_flag, aug_transformations)
maj_train_loader, maj_val_loader, maj_test_loader, maj_train_ds, maj_val_ds, maj_test_ds= pre_processing.load_data(maj_train_files_data, maj_val_files_data, maj_test_files_data, batch_size, augmentation_flag, aug_transformations)


all_KJ_loader, all_DC_loader, all_MH_loader, all_maj_loader, all_KJ_ds, all_DC_ds, all_MH_ds, all_maj_ds = InterOperator_utils.load_all_data(KJ_all_files_data, DC_all_files_data, MH_all_files_data, maj_all_files_data, batch_size, augmentation_flag, aug_transformations)

There are 246 files in total
n_train_files = 171
n_val_files = 49
n_test_files = 26
The training set is 70 percent of the dataset, the validation set is 20 percent of the dataset and the test set is 11 percent of the test set


Loading dataset: 100%|██████████| 246/246 [00:01<00:00, 239.52it/s]


#### Step 6: Average Fliess Kappa

In [23]:
avg_fk = InterOperator_utils.fliess_kappa(results_folder, KJ_test_loader, DC_test_loader, MH_test_loader, test_filenames)

fleiss kappa score for image 0 is 0.7463524233935485
fleiss kappa score for image 1 is 0.8688774997015362
fleiss kappa score for image 2 is 0.7975156294629788
fleiss kappa score for image 3 is 0.8988252137641005
fleiss kappa score for image 4 is 0.8702989567003989
fleiss kappa score for image 5 is 0.8922317287824635
fleiss kappa score for image 6 is 0.8179672693733938
fleiss kappa score for image 7 is 0.8284829442763697
fleiss kappa score for image 8 is 0.8752571007151269
fleiss kappa score for image 9 is 0.8220149590600276
fleiss kappa score for image 10 is 0.8708227876374188
fleiss kappa score for image 11 is 0.8735126415198144
fleiss kappa score for image 12 is 0.8737600899835141
fleiss kappa score for image 13 is 0.8991697325033952
fleiss kappa score for image 14 is 0.841318091839438
fleiss kappa score for image 15 is 0.8263852523177434
fleiss kappa score for image 16 is 0.8563459388824668
fleiss kappa score for image 17 is 0.908379000076869
fleiss kappa score for image 18 is 0.860

#### Step 7: Save overlay of segmentations on top of US image, preserving original image size

In [24]:
#Save overlay for train, validation, and test images
InterOperator_utils.save_image_overlay(results_subfolder = results_folder, files = test_files, n_classes = 5)
InterOperator_utils.save_image_overlay(results_subfolder = results_folder, files = val_files, n_classes = 5)
InterOperator_utils.save_image_overlay(results_subfolder = results_folder, files = train_files, n_classes = 5)

#### Step 8: Calculate Dice score, Hausdorff distance, and Jaccard index metrics for experts compared to the majority GT label.

In [25]:
InterOperator_utils.save_CLEAR_IOV_metrics(results_subfolder =results_folder, KJ_test_ds=KJ_test_ds, DC_test_ds=DC_test_ds, MH_test_ds=MH_test_ds, maj_test_ds=maj_test_ds, dice_metric = dice_metric, n_classes= n_classes, test_filenames=test_filenames)

the prediction of class 1 is all 0, this may result in nan/inf distance.
the prediction of class 1 is all 0, this may result in nan/inf distance.
the prediction of class 1 is all 0, this may result in nan/inf distance.
the prediction of class 4 is all 0, this may result in nan/inf distance.
the prediction of class 4 is all 0, this may result in nan/inf distance.
